# Welcome to HAM Tool Box!

This ipython notebook aims to help you to do your first steps with HAM (or as a spreadsheet for HAM). 

In this tutorial we will explain how to: 
* [set up](#set) an HAM analysis using the different availble options.
* use the different [queries](#query) that allow to fetch the information you want.
* explore what the ham [objects](#object) have to offer.
* [compare](#compare) several genomes throught their HOGS.
* run [hogvis](#hvis) (single hog visualisation)
* run [treeprofile](#tprofile) (species tree with annotated node with evolutionary events)



#### <a id='set'></a>
## SET UP THE HAM

### Required import

In [ ]:
#  This is the HAM package
import pyham

#  OPTIONAL: only if you want to have the logger information printed
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

### First, we need a species tree and an orthoXML !

In [ ]:
import os, inspect # This is just to get the example file from your pyham package installation

#  Select a nwk file as a taxonomy reference
nwk_path = os.path.dirname(inspect.getfile(pyham)) + "/../tests/data/simpleEx.nwk"

#  And extract the newick tree as a string
tree_str = pyham.utils.get_newick_string(nwk_path, type="nwk")

# Then you select your favorite orthoXML file
orthoxml_path = os.path.dirname(inspect.getfile(pyham)) + "/../tests/data/simpleEx.orthoxml"


### Let's feed HAM with some HOGs and a tree.

In [ ]:
# pyham.Ham is the main object that containes all information and functionalities.
ham_analysis = pyham.Ham(tree_str, orthoxml_path, use_internal_name=True)


### Alternative options: filtering and taxonomic range naming!

#### Taxonomic range naming
If you don't want to use the internal names defined by the newick specie tree (or if it's have support values as internal names) you can use the automatic internal node naming of HAM (i.e. concatenation of children names).


In [ ]:
# the use_internal_name attribute is doing the job
ham_analysis_no_name = pyham.Ham(tree_str, orthoxml_path, use_internal_name=False)

# In the previous section we use the internal name of the newick tree:
print("Ancestral genomes name using newick internal names:")
for ag in ham_analysis.taxonomy.internal_nodes:
    print("\t- {}".format(ag.name))
    

In [ ]:
# Here we use the artificial internal names built by pyham:
print("Ancestral genomes name using artificial ham names:")
for ag in ham_analysis_no_name.taxonomy.internal_nodes:
    print("\t- {}".format(ag.name))
    

In [ ]:
#  In case you are using use_internal_name=False, the following function returns an ascii representation
#  of the newick tree with  articial names. This can be used even before instanciating the HAM object which
#  can be useful to avoid stupid error of query by name in scripts.
print(pyham.utils.previsualize_taxonomy(tree_str))


#### Pre Parsing filtering
If you have a large orthoXML file or you only want parse information of interest you can use the filter option.

In [ ]:
# First you instanciate an empty ParserFilter object
filter_ham = pyham.ParserFilter()

# Then you can add to the filter hogs of interest using the following function (those 3 examples are doing the same)
filter_ham.add_hogs_via_hogId([2]) # by its toplevel hog id 
filter_ham.add_hogs_via_GeneIntId([2]) # by a gene unique id that belong to the hog of interest 
filter_ham.add_hogs_via_GeneExtId(["HUMAN2"]) # by a gene external id that belongs to the hog of interest 

# Finaly, you set up the HAM object as before using the filter_object attribute
ham_analysis_filter = pyham.Ham(tree_str, orthoxml_path, use_internal_name=True, filter_object=filter_ham)

# If you look at the log information beneath it is different from the one previously printed !!

<a id='query'></a>
## QUERIES: HOW TO TALK WITH HAM ?

We can separate queries into the following categories:
*  [ExtantGene](#xgeneq)
*  [HOG](#hogq)
*  [ExtantGenome](#xgenomeq)
*  [AncestralGenome](#ancgenomeq)
*  [Taxon](#taxq)

<a id='xgeneq'></a>
### ExtantGene queries

ExtantGene are the object to represent genes at the leaves.

In [ ]:
# Get a gene by its unique (orthoxml) id
gene_human3 = ham_analysis.get_gene_by_id(3)
print(gene_human3)

# Get a list of genes that match an external id
potentential_genes_human3 = ham_analysis.get_genes_by_external_id("HUMAN3")
print(potentential_genes_human3[0])

## You should see twice the same gene printed below!

# You can also get all genes created as a list
list_genes = ham_analysis.get_list_extant_genes()

# or as the dictionnary (key <-> unique id and value <-> gene)
dict_genes = ham_analysis.get_dict_extant_genes()

<a id='hogq'></a>
### HOG queries

In [ ]:
# Get a HOG by its top level unique id
HOG_3 = ham_analysis.get_hog_by_id(3)
print(HOG_3)

HOG_3 = ham_analysis.get_hog_by_gene(gene_human3)
print(HOG_3)

## You should see twice the same HOG printed beneath !


In [ ]:

# You can also get all genes created as a list
list_toplevel_hogs = ham_analysis.get_list_top_level_hogs()

# or as the dictionnary (key <->  top level id and value <-> HOG)
dict_toplevel_hogs = ham_analysis.get_dict_top_level_hogs()

<a id='xgenomeq'></a>
### ExtantGenome queries

In [ ]:
# Get a extant genome by its name
genome_human = ham_analysis.get_extant_genome_by_name("HUMAN")
print(genome_human)


In [ ]:

# You can also get all extant genomes created as a list
list_genome = ham_analysis.get_list_extant_genomes()
print("List of species:")
for g in list_genome:
    print("\t- {}".format(g.name))

<a id='ancgenomeq'></a>
### AncestralGenome queries

AncestralGenome are the object to represent the genome associate to internal node in the species tree.

In [ ]:
# Get an ancestral genome by its name
genome_rodents_1 = ham_analysis.get_ancestral_genome_by_name("Rodents")
print(genome_rodents_1)

# And in case you specified use_internal_name=False ! 
genome_rodents_2 = ham_analysis_no_name.get_ancestral_genome_by_name("MOUSE/RATNO")
print(genome_rodents_2)

In [ ]:
# Get an ancestral genome by looking at the mrca of 2+ genomes
# First you get the descendant genomes as we seen above
genome_rat = ham_analysis.get_extant_genome_by_name("RATNO")
genome_mouse = ham_analysis.get_extant_genome_by_name("MOUSE")
# Then you get the corresponding mrca ancestral genomes 
genome_rodents_3 = ham_analysis.get_ancestral_genome_by_mrca_of_genome_set({genome_rat, genome_mouse})
print(genome_rodents_3)

In [ ]:
# You can also get an ancestral genome by its taxonomy node
taxon = ham_analysis.get_taxon_by_name("Rodents") # as seen in the next section
genome_rodents_4 = ham_analysis.get_ancestral_genome_by_taxon(taxon)
print(genome_rodents_4)

In [ ]:
# You can also get all ancestral genomes created as a list
list_genome = ham_analysis.get_list_ancestral_genomes()
print("\n List of ancestral genomes:")
for g in list_genome:
    print("\t- {}".format(g.name))

In [ ]:
# You can also get all ancestral genomes created as a list
list_genome = ham_analysis_filter.get_list_ancestral_genomes()
print("\n Here for the filter version of HAM, the list of ancestral genomes:")
for g in list_genome:
    print("\t- {}".format(g.name))
    
print("'Vertebrate' is not present in the filter list because not required during the parsing")

<a id='taxq'></a>
### Taxon queries

In [ ]:
# You can get an Taxonomy node by its name
taxon = ham_analysis.get_taxon_by_name("Rodents") 
print(taxon.name)

taxon = ham_analysis.get_taxon_by_name("HUMAN") 
print(taxon.name)

<a id='object'></a>
## HAM OBJECT: SHOW ME YOUR SECRETS?

Let's have a look at each HAM object !
*  [Gene/HOG](#gdef)
*  [Genome](#genomedef)
*  [Taxonomy](#taxonomydef)


<a id='gdef'></a>
### Gene/HOG

#### AbstractGene (Both ExtantGene and HOG)

In [ ]:
# we select one extant gene and one hog for our demo
print("Demo| gene/hog: ")
gene2 = ham_analysis.get_gene_by_id(2)
print("\t - gene 2: ", gene2)
hog1 = ham_analysis.get_hog_by_id(1)
print("\t - hog 1: ", hog1)

###### The cool shared features between both of them are:

In [ ]:
# that we can fetch the genome where they belong to
print("Demo| gene/hog genome: ")
print("\t - gene 2: ", gene2.genome)
print("\t - hog 1: ", hog1.genome)
print("\n")

In [ ]:
# that we can get the related top level hog
print("Demo| gene/hog related top level hog: ")
print("\t - gene 2: ", gene2.get_top_level_hog())
print("\t - hog 1: {} (it  returns itself it already top level hog) ".format(hog1.get_top_level_hog()))
print("\n")

In [ ]:
# that we can get the related hogs at a specific level 
rodents = ham_analysis.get_ancestral_genome_by_name("Rodents")
print("Demo| gene/hog related top level hog: ")
print("\t - gene 2: ", gene2.get_at_level(rodents))
print("\t - hog 1: ", hog1.get_at_level(rodents))
print("/!\ THIS CAN RETURN MORE THAN ONE HOG /!\ ")
hog3 = ham_analysis.get_hog_by_id(3)
print("\t - hog 3: ", hog3.get_at_level(rodents))
print("\n")

In [ ]:
# Singleton -> a gene present in a species of the orthoxml but that doesn't belong to any hog.

# that we can know if the abstractGene is a singleton or not
human_5 = ham_analysis.get_gene_by_id("5")
print("Demo| gene/hog singletons ? : ")
print("\t - gene 2: {}".format(gene2.is_singleton()))
print("\t - gene 5: {}".format(human_5.is_singleton()))
print("\t - HOG 1: {}".format(hog1.is_singleton()))
print("\n")

In [ ]:
# that we can get the ancestral HOG of an abstractGene at a specific level. In addition a boolean specify if a duplication occured in between the two levels.
mamm = ham_analysis.get_ancestral_genome_by_name("Mammalia")
hog3_rodents = hog3.get_at_level(rodents)[0]

print("Demo| gene ancestor hog at Mammalia level: ")
print("\t - gene 2: {}".format(gene2.search_ancestor_hog_in_ancestral_genome(mamm)))
print("\t - hog 3 at rodents: {}".format(hog3_rodents.search_ancestor_hog_in_ancestral_genome(mamm)))

#### ExtantGene

In [ ]:
# You can get a dictionary with all the cross references for a gene
gene2 = ham_analysis.get_gene_by_id(2)
print(gene2.get_dict_xref())

#### HOG

###### The cool feature in for HOG are that you can:

In [ ]:
# demo hog
hog1 = ham_analysis.get_hog_by_id(1)
hog3 = ham_analysis.get_hog_by_id(3)

In [ ]:
# get all descendant genes
desc_genes = hog1.get_all_descendant_genes()
print(desc_genes)

In [ ]:
# get all descendant genes clustered by species
desc_genes_clustered = hog3.get_all_descendant_genes_clustered_by_species()
for species, genes in desc_genes_clustered.items():
    print(species, genes)

In [ ]:
# get all descendant level (internal node ancestral genomes)
desc_level = hog1.get_all_descendant_hog_levels()
for genome in desc_level:
    print(genome.name)

In [ ]:
# get all descendant hogs
desc_hog = hog1.get_all_descendant_hogs()
print(desc_hog)

In [ ]:
# visit the hog with prefix, postfix, leaf callback function
# (reading the docs at pyham.abstract.HOG.visit() is required here) 

# -----------------------------------------------------------#
# -- This is an example to apply a function to each leaves --#
# -----------------------------------------------------------#
def print_and_append_leaf(current, child, list):
    list.append(child)
    print(child)
    return list

passed_object = []
return_object = hog1.visit(passed_object, function_extant_gene=print_and_append_leaf)

print("The return object (list of leaves): {}".format(return_object))

In [ ]:
# ------------------------------------------------------------------#
# -- This is an example to apply a function to each node (prefix) --#
# ------------------------------------------------------------------#
def print_and_append_node(current, list):
    list.append(current)
    print(current)
    return list

passed_object = []
return_object = hog1.visit(passed_object, function_prefix=print_and_append_node)

print("The return object (list of internale nodes): {}".format(return_object))

In [ ]:
# -------------------------------------------------------------------------------------------#
# -- This is an example to apply a function to each node after the recursive call (prefix) --#
# -------------------------------------------------------------------------------------------#
def print_and_append_node(self, child, elem):
    elem.append(child)
    print(child)
    return elem

passed_object = []
return_object = hog1.visit(passed_object, function_postfix=print_and_append_node)

print("The return object (list of internale nodes child): {}".format(return_object))

<a id='genomedef'></a>
### Genome


In [ ]:
human_genome = ham_analysis.get_extant_genome_by_name("HUMAN")
rodents_genome = ham_analysis.get_ancestral_genome_by_name("Rodents")

# You can get the name
print("Get genome name:")
print("\t -{}".format(human_genome.name))
print("\t -{}".format(rodents_genome.name))
print("\n")

In [ ]:
# You can get the related node in the taxonomy
print("Get genome taxon:")
print(human_genome.taxon)
print(rodents_genome.taxon)
print("\n")

In [ ]:
# You can get the list of genes associated to this genomes
print("Get genome genes:")
print(human_genome.genes)
print(rodents_genome.genes)
print("\n")

In [ ]:
# You can get the number of genes associated to this genomes
print("Get genome genes number:")
print(human_genome.get_number_genes(singleton=True))
print(human_genome.get_number_genes(singleton=False)) # Here we are not counting the singletons as species genes !
print(rodents_genome.get_number_genes())
print("\n")

In [ ]:
for h, gs in rodents.get_ancestral_clustering().items():
    print("HOG: {} -> genes: {}".format(h,gs))

<a id='taxonomydef'></a>

### Taxonomy

The pyham.taxonomy.Taxonomy object contains all informations about the species tree structure.

In [ ]:
# it contains all the leaves nodes:
print(ham_analysis.taxonomy.leaves)
print("\n")

In [ ]:
# and the internal node:
print(ham_analysis.taxonomy.internal_nodes)
print("\n")

In [ ]:
# The main interest of this object is the taxonomy.tree object.
tree = ham_analysis.taxonomy.tree

# Since it's an ete3.Etree tree is contains all the built-in functionalities
print("GenomeName leaf? root?")
print("-----------------------")
for node in ham_analysis.taxonomy.tree.traverse(): # .traverse() is an ete3.etree method.
    print(node.name, node.is_leaf(), node.is_root())

<a id='compare'></a>
## COMPARE SEVERAL GENOMES

In HAM, you can compare genomes based on the evolutionary history of their genes. 

### Vertical comparison
For example you can compare the human genome with its ancestor at the level of Vertebrates. This means that you will investigate on how the ancestral genes (HOGs) in the Vertebrates ancestral genome have evolved (did they stay single copy, have duplicated or been lost, or did they evolve some new genes?) to gave rise to their related descendants in the human genome.

The comparison is not restricted to a extant genome and its ancestral genome, you could also compare 2 ancestral genomea that are on the same lineage (e.g the rodents with the mammals).

As show in the following example:

In [ ]:
# Get the genome of interest
human = ham_analysis.get_extant_genome_by_name("HUMAN")
vertebrates = ham_analysis.get_ancestral_genome_by_name("Vertebrata")

# Instanciate the gene mapping !
vertical_human_vertebrates = ham_analysis.compare_genomes_vertically(human, vertebrates) # The order doesn't matter!



Ask the mapper about the evolutionary history between the human and the vertebrates genes.

In [ ]:
# The identical genes (that stay single copies) 
print("HOG at vertebrates -> descendant gene in human")
print(vertical_human_vertebrates.get_identical())
print("\n")

In [ ]:
# The duplicated genes (that have duplicated) 
print("HOG at vertebrates -> list of descendants gene in human")
print(vertical_human_vertebrates.get_duplicated())
print("\n")

In [ ]:
# The gained genes (that emerged in between)
print("List of human gene")
print(vertical_human_vertebrates.get_gained())
print("\n")

In [ ]:
# The lost genes (that been lost in between) 
print("HOG at vertebrates that are lost")
print(vertical_human_vertebrates.get_lost())
print("\n")

### Lateral comparison
An other example could be to compare the rodents ancestral genome vs the primates ancestral genome through their common ancestor at the level of Euarchontoglires. Similar to the last paragraph you are investigating on how the ancestral Euarchontoglires genes gave rise to their related descendant ancestrals genomes in both primates and rodents.

In [ ]:
# Get the genome of interest
human = ham_analysis.get_extant_genome_by_name("HUMAN")
mouse = ham_analysis.get_extant_genome_by_name("RATNO")

# Instanciate the gene mapping !
lateral_human_mouse = ham_analysis.compare_genomes_lateral(human, mouse) # The order doesn't matter!


 Ask the mapper about the genes evolutionary history between the human and the rat with and the Euarchontoglires (their mrca).

In [ ]:
# The identical genes (that stay single copies) 
print("IDENTICAL GENES")
for hogs, dict_genome_gene in lateral_human_mouse.get_identical().items():
    print("\t- HOG at Euarchontoglires {} is the ancestor of: ".format(hogs))
    for g, gene in dict_genome_gene.items():
        print("\t\t-  {} in {}".format(gene, g))
print("\n")

In [ ]:
# The duplicated genes (that have duplicated) 
print("DUPLICATED GENES")
for hogs, dict_genome_genes in lateral_human_mouse.get_duplicated().items():
    print("\t- HOG at Euarchontoglires {} is the ancestor of: ".format(hogs))
    for g, genes in dict_genome_gene.items():
        print("\t\t-  {} in {}".format(genes, g))
print("\n")

In [ ]:
# The gained genes (that emerged in between)
print("GAINED GENES")
for genome, gains in lateral_human_mouse.get_gained().items():
    print("\t- Genome {} have gained:".format(genome))
    for g in gains:
        print("\t\t-  {}".format(g))
print("\n")

In [ ]:
# The lost genes (that been lost in between) 
print("LOST GENES")
for hog, genomes in lateral_human_mouse.get_lost().items():
    print("\t- HOG at Euarchontoglires {} have been lost in ".format(hog))
    for g in genomes:
        print("\t\t- {}".format(g))
print("\n")

<a id='hvis'></a>
## HOGVIS

Hogvis is a tool to visualise how the HOG members genes are clustering based on their ancestral genes membership. 

In other words, given a taxonomic range of interest the HOG extant genes are grouped based on their ancestral genes at the descending taxonomic range.

**Let's run the following portion of code to see and example with its description below:**  

In [ ]:
hog_3 =  ham_analysis.get_hog_by_id(3);ham_analysis.create_hog_visualisation(hog=hog_3,outfile="HOG{}.html".format(hog_3.hog_id));from IPython.display import IFrame;IFrame("HOG{}.html".format(hog_3.hog_id), width=700, height=350)

Hogvis is composed of two part:
- **Species tree**: This part allows you to select the level of interest (click to freeze the hogvis at this level)
- **Genes panel**: Each line represents an extant genome meanwhile each square represents an extant gene.

If you look at the level of Vertabrata (hover the node), you will see all the genes that are descendant from one single ancestral gene at this level.

If you look now at the level of Euarchontoglires (again, hover the node), you can see that genes are split by a vertical line. This line is used to separate ancestral gene groups. Each columns created by those vertical separators are representing an ancestal gene at the level of interest and contains all the extant genes that are descending from it. 

Here we can observe one column at the level of Mammalia and 2 columns at the level of Euarchontoglires which mean that a duplication had occured in between those two levels.

### Hogvis : single HOG

In [ ]:
# Select an HOG
hog_2 =  ham_analysis.get_hog_by_id(2)

# create the hogvis for it and store it into an html file
output_name = "HOG{}.html".format(hog_2.hog_id)
ham_analysis.create_hog_visualisation(hog=hog_2,outfile=output_name)

# Here a little demo of what you can see with hogs vis
from IPython.display import IFrame
IFrame(output_name, width=700, height=350)

### Hogvis : single HOG at a specific taxonomic range (ancestral genome)

In [ ]:
# select an HOG
hog_3 =  ham_analysis.get_hog_by_id(3)

# then an genome
mammals = ham_analysis.get_ancestral_genome_by_name("Mammalia")

# and you get the related HOG for this genome
hog3_Mammalia = hog_3.get_at_level(mammals)[0] # this function returns a list, I select the first and only one.

# create the hogvis for it and save it into an html file
output_name = "HOG{}.html".format(hog3_Mammalia.hog_id)
ham_analysis.create_hog_visualisation(hog=hog3_Mammalia,outfile=output_name)

# Here a little demo of what you can see with hogs vis
from IPython.display import IFrame
IFrame(output_name, width=700, height=350)

### Hogvis : all HOGs for a specific taxonomic range (ancestral genome)

In [ ]:
#  Select a genome
mammals = ham_analysis.get_ancestral_genome_by_name("Mammalia")

# Iterate over all its HOGs
for hog in mammals.genes:
    # and create the hogvis for each
    ham_analysis.create_hog_visualisation(hog=hog,outfile="HOG-mammals{}.html".format(hog.hog_id))

<a id='tprofile'></a>
## TREEPROFILE

TreeProfile is a tool to visualise how the genes have evolved in terms of evolutionnary events alonge a phylogenetic tree (duplication, lost, gained).

**ATTENTION: Rendering the tree profile picture requires the PyQt4 library to be installed. We provide in the documentation a section to help you to install it!**

**Let's run the following portion of code to see and example with its description below:**  

In [ ]:
treeprofile = ham_analysis.create_tree_profile(export_with_histogram=True, outfile="tp.png");from IPython.display import Image;Image(filename='tp.png') 

TreeProfile is composed of :
- **Species tree**: This is the reference taxonomy used by HAM.
- **Internal node histogram**: Represent the proportion of genes at this level that are emerged from their ancestor in the parent genome by one of the 4 following types of evolutionary events: identicals (identical in terms of numbers, duplicated, lost at this level, gained at this level) . In additions, the "Genes" bar displays the total number of genes of a related genome.
- **Legend**: Provides a description of the histogram bar and the scale.

### TreeProfile on the whole HAM genomic setup 

Severals options are available:
- **outfile**: if specify the visualisation will be exporte as image and save in this file. The output file extension will define the format (.SVG, .PDF, .PNG).
- **export_with_histogram**: If you don't want the histogram you can set this argument to False and raw text will be displayed instead of histogram.


In [ ]:
# create the treeprofile object
treeprofile = ham_analysis.create_tree_profile(export_with_histogram=True, outfile="tp.png")

#  This is just to display it here
from IPython.display import Image
Image(filename='tp.png') 

### TreeProfile for a specific HOG

In addition to the previous parameters, if you provide a single HOG the treeprofile can be build only on using the information of that particular hog and will represent the evolutionary history of this HOG in terms of duplication, loss and identical genes accross each taxonomic range.


In [ ]:
# select an HOG
hog_3 =  ham_analysis.get_hog_by_id(3)

# create the treeprofile object
treeprofile_hog_3 = ham_analysis.create_tree_profile(hog=hog3, export_with_histogram=True, outfile="tph3.png")

#  This is just to display it here
from IPython.display import Image
Image(filename='tph3.png') 